In [182]:
import joblib
import pandas as pd
from datetime import timedelta

def test_load_dfs(df_1,df_2):
    template_1 = joblib.load('tests/loans_template.pkl').dtypes
    template_3 = joblib.load('tests/recharges_template.pkl').dtypes
    assert (df_1.dtypes == template_1).all()
    assert (df_2.dtypes == template_3).all()

    return df_1,df_2


def load_dfs(paths=[
    'Brazil_DS_loans_2019-11-10_2019-12-05.csv',
    'Brazil_DS_prev_loans.csv',
    'Brazil_DS_recharges_2019-08-10_2019-12-05.csv']):
    
    loans_actual = pd.read_csv(paths[0],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['created_at','paid_at'])
    loans_prev = pd.read_csv(paths[1],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['created_at','paid_at'])
    loans = pd.concat([loans_actual,loans_prev],ignore_index=True)
    recharges = pd.read_csv(paths[2],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['recharge_timestamp'])
    #return  test_load_dfs(loans, recharges)
    return  loans, recharges


loans, recharges = load_dfs()

# Falta fazer

* Otimização dos parametros com k-fold
* Criar os cenários para escolher qual melhor modelo
* Criar os mocks de teste do feature eng
* Relátorio final
* Se der tempo fazer um CLI básico

# Exploratory data analysis

É interessante observar que os emprestimos não pagos não varia muito da mediana populacional, 5, entretanto utilizar essa informação pode gerar bias no modelo preditivo.


LEMBRETES:
    Criar intervalos de loans permitidos para cada usuário
    Fazes dois mocks para as funções de feature eng

Notas:
    Distribuição acumulado inad. 
        target_sum
        0    7044
        1    2463
        2       5
        4       1

#t_loans


In [22]:
#print('General Loans stats \n',t_loans['amount'].describe(),'\n',f"Median : {t_loans['amount'].median()}")
#print('Paid stats \n',t_loans.query('target == 0').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())
#print('Not Paid stats \n',t_loans.query('target == 1').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())

In [179]:
loans['created_at'].max()

Timestamp('2019-12-04 23:54:57.280521')

In [181]:
loans.columns

Index(['uuid', 'loan_id', 'amount', 'created_at', 'paid_at',
       'paid_days_interval', 'target'],
      dtype='object')

In [218]:
loans['created_at'].max()

Timestamp('2019-12-04 23:54:57.280521')

In [239]:
from dataclasses import dataclass
from typing import List
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from datetime import datetime


@dataclass
class FeatureEngineering:
    train_loans: pd.DataFrame
    test_loans: pd.DataFrame
    train_recharges: pd.DataFrame
    days_to_default:int = 60
    

    def __post_init__(self):

        self.train_loans = self.__feature_eng_loans(self.train_loans.copy())
        self.test_loans = self.__feature_eng_loans(self.test_loans.copy())
        self.train_recharges = self.__feature_eng_recharges(self.train_recharges.copy())
        

    def __feature_eng_loans(self,df_l: pd.DataFrame) -> pd.DataFrame:
            '''
            Feature engineering for loans dataframe using historical data
            already_default: 0 if not defaulted, 1 if defaulted
            sum_amoun: sum amount of all previous loans paid
            count_loans: number of previous loans paid

            :param df_l: loans dataframe
            :return: feature engineered loans dataframe
            '''
            #calc days interval between dates
            df_l['paid_days_interval'] = (df_l['paid_at'] - df_l['created_at'])
            df_l['target'] = df_l['paid_days_interval'].apply(lambda x: 1 if x.days > self.days_to_default else 0)

            already_default = df_l.groupby('uuid').sum()['target'].apply(lambda x: 1 if x > 0 else 0)
            sum_amount = df_l.query('target == 0').groupby('uuid').sum()['amount']
            count_loans = df_l.query('target == 0').groupby('uuid').count()['amount']

            out_df = pd.DataFrame(df_l.groupby('uuid').count().index)
            out_df = out_df.join(already_default,on='uuid',how='left')
            out_df = out_df.join(sum_amount,on='uuid',how='left',rsuffix='_sum')
            out_df = out_df.join(count_loans,on='uuid',how='left',rsuffix='_count')
            out_df.columns = ['uuid','target','sum_amount','count_loans']
            return out_df
        
    def __feature_eng_recharges(self,df_r: pd.DataFrame) -> pd.DataFrame:
        '''
        Feature engineering for recharges dataframe using historical data
        freq_recharges_weekly: mean frequency recharges per week
        recharges_weekly: median frequency recharges per week
        delta_after_recharge: difference between balance after recharge and recharge value

        :param df_r: recharges dataframe
        :return: feature engineered recharges dataframe
        '''
        df_r['delta_after_recharge'] = df_r['balance_after_recharge'] - df_r['recharge_value']
        df_r['back_recharge_timestamp'] = pd.to_datetime(df_r['recharge_timestamp']) - pd.to_timedelta(7, unit='d')
        max_date = df_r['back_recharge_timestamp'].max()
        min_date = df_r['back_recharge_timestamp'].min()
        count_weeks = (max_date - min_date).days // 7
        weekly_df = df_r \
            .groupby(['uuid', pd.Grouper(key='recharge_timestamp', freq='W-MON')]) \
            .count() \
            .groupby('uuid')
        
        freq_recharges_weekly = weekly_df.sum()['recharge_value']/count_weeks
        recharges_weekly = weekly_df.median()['recharge_value']/count_weeks
        delta_after_recharges = df_r.groupby('uuid').median()['delta_after_recharge']
        
        out_df = pd.DataFrame(df_r.groupby('uuid').count().index)
        out_df = out_df.join(freq_recharges_weekly,on='uuid',how='left',rsuffix='_median')
        out_df = out_df.join(recharges_weekly,on='uuid',how='left',rsuffix='_median')
        out_df = out_df.join(delta_after_recharges,on='uuid',how='left')
        out_df.columns = ['uuid','freq_recharges_weekly','recharges_weekly','delta_after_recharges']
        return out_df

    @staticmethod
    def remove_perfect_correlation(df: pd.DataFrame,max_cor = 0.95) -> pd.DataFrame:
        ''''
        Remove columns with perfect correlation with other columns
        :param df: dataframe
        :return: dataframe without columns with perfect correlation
        
        corr_matrix = df.corr()
        corr_matrix.loc[:, :] = np.tril(corr_matrix.values, k=-1)
        cols_to_drop = corr_matrix.loc[:,(corr_matrix.abs() > max_cor).any()]
        print((corr_matrix.abs() > max_cor).any())
        df = df.drop(cols_to_drop,axis=1)
        TODO
        '''
        df = df.drop('count_loans',axis=1)
        return df

@dataclass
class ForecastDefault:
    '''
    docstring
    '''
    loans_hist: pd.DataFrame
    recharges_hist: pd.DataFrame
    estimators_list: List
    inicial_date: str = '2000-01-01'
    limit_date: str = '2019-12-05'
    days_to_default: int = 60
    fill_na:bool = True

    def __post_init__(self):
        date_format = '%Y-%m-%d'
        self.inicial_date = datetime.strptime(self.inicial_date,date_format)
        self.limit_date = datetime.strptime(self.limit_date,date_format)
        self.loans_hist['paid_at'] = self.loans_hist['paid_at'].fillna(self.limit_date + timedelta(days=1))
        last_date = self.limit_date - timedelta(days=self.days_to_default)
        

        self.loans = self.loans_hist[self.loans_hist['created_at'] > self.inicial_date].copy()
        self.recharges = self.recharges_hist[self.recharges_hist['recharge_timestamp'] > self.inicial_date].copy()
        

        train_loans = self.loans[self.loans['created_at'] < last_date]
        test_loans = self.loans[((self.loans['created_at'] > last_date) & (self.loans['created_at'] < self.limit_date))]
        train_recharges = self.recharges[self.recharges['recharge_timestamp'] < last_date]
        
        print('last_date',last_date)
        print('limit_date',test_loans['created_at'].max())

        
        print('train_loans: ',train_loans.shape)
        print('test_loans: ',test_loans.shape)


        fe = FeatureEngineering(train_loans,test_loans,train_recharges,days_to_default=self.days_to_default)

        train_df = fe.train_loans.merge(fe.train_recharges, on='uuid', how='left')
        train_df = fe.remove_perfect_correlation(train_df)

        if self.fill_na:
            train_df.fillna(0,inplace=True)

        self.model = FitModel(train_df,fe.test_loans[['uuid','target']],self.estimators_list)



from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

@dataclass
class FitModel:
    '''
    Class to fit the model
    :param train_df: dataframe with training data
    :param fe_test_df: dataframe containing forward targets to evaluate performance metrics
    :param estimator_list: list of estimators to use in the model
    '''
    train_df: pd.DataFrame
    fe_test_df: pd.DataFrame
    estimators_list: List


    def __post_init__(self):
        self.train_X, self.train_y = self.__under_sampling(self.train_df.copy())
        self.test_df = self.train_X.merge(self.fe_test_df,on='uuid',how='inner')
        

        self.train_X.drop('uuid',axis=1,inplace=True)
        self.test_df.drop('uuid',axis=1,inplace=True)

        self.test_X = self.test_df.drop('target',axis=1)
        self.test_y = self.test_df['target']

        print('train_X: ',self.train_X.shape)
        print('train_y: ',self.train_y.shape)
                

        #Vai mudar para aceitar varios estimadores mas agora vai sobreescrever o modelo
        for est in self.estimators_list:
            self.est = self.__fit(est)


        self.evaluate_model()
        
    def evaluate_model(self):
        print('Confusion Matrix: \n',confusion_matrix(self.test_y, self.predict(self.test_X)))
        print('Classification Report: \n',classification_report(self.test_y, self.predict(self.test_X)))
    
    def predict(self,X: pd.DataFrame) -> pd.Series:
        '''
        Predict using the fitted model
        :param X: dataframe with features
        :return: predictions
        '''
        #pred_df = pd.DataFrame(X.index)
        #pred_df['default_probability'] = self.est.predict(X)
        pred_df = self.est.predict(X)
        return pred_df


    def __fit(self,estimator):
        '''
        docstring
        '''
        est = estimator()
        est.fit(self.train_X, self.train_y)
        return est

    
    def __under_sampling(self,df: pd.DataFrame) -> pd.DataFrame:
        '''
        Under sampling of dataframe
        :param df: dataframe
        :return: undersampled dataframe
        '''
        rus = RandomUnderSampler(random_state=42)
        train_X, train_y = rus.fit_resample(
            train_df.drop(['target'],axis=1),train_df['target'])
        return train_X, train_y



s = ForecastDefault(loans_hist=loans, recharges_hist=recharges, estimators_list=[RandomForestClassifier],inicial_date='2019-01-01',days_to_default=1,limit_date='2019-12-05')
    

last_date 2019-12-04 00:00:00
limit_date 2019-12-04 23:54:57.280521
train_loans:  (18336, 5)
test_loans:  (625, 5)
train_X:  (238, 5)
train_y:  (238,)
Confusion Matrix: 
 [[0 0]
 [8 8]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



/home/enniobastos/miniconda3/envs/juvo_test_pycaret/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
PREDICT_FORWARD_DAYS = 60
last_date = loans['created_at'].max() - timedelta(days=60)
recharges['delta_after_recharge'] = recharges['balance_after_recharge'] - recharges['recharge_value']
loans['paid_days_interval'] = (loans['paid_at'] - loans['created_at']).dt.days.fillna(61)
loans['target'] = loans['paid_days_interval'].apply(lambda x: 1 if x > 60 else 0)

train_loans = loans[loans['created_at'] < last_date].copy()
test_loans = loans[loans['created_at'] > last_date].copy()


train_recharges = recharges[recharges['recharge_timestamp'] < last_date].copy()
test_recharges = recharges[recharges['recharge_timestamp'] > last_date].copy()


def feature_eng_loans(loans_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Feature engineering for loans dataframe using historical data
    already_default: 0 if not defaulted, 1 if defaulted
    sum_amoun: sum amount of all previous loans paid
    count_loans: number of previous loans paid
    '''
    already_default = loans_df.groupby('uuid').sum()['target'].apply(lambda x: 1 if x > 0 else 0)
    sum_amount = loans_df.query('target == 0').groupby('uuid').sum()['amount']
    count_loans = loans_df.query('target == 0').groupby('uuid').count()['amount']

    out_df = pd.DataFrame(loans_df.groupby('uuid').count().index)
    out_df = out_df.join(already_default,on='uuid',how='left')
    out_df = out_df.join(sum_amount,on='uuid',how='left',rsuffix='_sum')
    out_df = out_df.join(count_loans,on='uuid',how='left',rsuffix='_count')
    out_df.columns = ['uuid','target','sum_amount','count_loans']
    return out_df

def feature_eng_recharges(recharges_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Feature engineering for recharges dataframe using historical data
    freq_recharges_weekly: mean frequency recharges per week
    recharges_weekly: median frequency recharges per week
    delta_after_recharge: difference between balance after recharge and recharge value
    '''
    recharges_df['back_recharge_timestamp'] = pd.to_datetime(recharges_df['recharge_timestamp']) - pd.to_timedelta(7, unit='d')
    max_date = recharges_df['back_recharge_timestamp'].max()
    min_date = recharges_df['back_recharge_timestamp'].min()
    count_weeks = (max_date - min_date).days // 7
    weekly_df = recharges_df \
        .groupby(['uuid', pd.Grouper(key='recharge_timestamp', freq='W-MON')]) \
        .count() \
        .groupby('uuid')
    
    freq_recharges_weekly = weekly_df.sum()['recharge_value']/count_weeks
    recharges_weekly = weekly_df.median()['recharge_value']/count_weeks
    delta_after_recharges = recharges_df.groupby('uuid').median()['delta_after_recharge']
    
    out_df = pd.DataFrame(recharges_df.groupby('uuid').count().index)
    out_df = out_df.join(freq_recharges_weekly,on='uuid',how='left',rsuffix='_median')
    out_df = out_df.join(recharges_weekly,on='uuid',how='left',rsuffix='_median')
    out_df = out_df.join(delta_after_recharges,on='uuid',how='left')
    out_df.columns = ['uuid','freq_recharges_weekly','recharges_weekly','delta_after_recharges']
    return out_df

test_loans_fe = feature_eng_loans(test_loans)

train_loans_fe = feature_eng_loans(train_loans)
train_recharges_fe = feature_eng_recharges(train_recharges)
train_df = train_loans_fe.merge(train_recharges_fe,on='uuid',how='left')

#check nulls
print('NULLS :',train_df.isnull().sum())
train_df.fillna(0,inplace=True)
print('NA FILLED :',train_df.isnull().sum())

NULLS : uuid                      0
target                    0
sum_amount               79
count_loans              79
freq_recharges_weekly    34
recharges_weekly         34
delta_after_recharges    34
dtype: int64
NA FILLED : uuid                     0
target                   0
sum_amount               0
count_loans              0
freq_recharges_weekly    0
recharges_weekly         0
delta_after_recharges    0
dtype: int64


In [24]:
train_df.describe()

,target,sum_amount,count_loans,freq_recharges_weekly,recharges_weekly,delta_after_recharges
count,1033.000000,1033.000000,1033.000000,1033.000000,1033.000000,1033.000000
mean,0.115198,12.662149,2.323330,0.956559,0.170196,0.541723
std,0.319416,14.921600,2.280692,0.824489,0.087977,0.964410
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,5.000000,1.000000,0.375000,0.125000,0.075000
50%,0.000000,10.000000,2.000000,0.750000,0.125000,0.285000
75%,0.000000,15.000000,3.000000,1.250000,0.250000,0.670000
max,1.000000,150.000000,20.000000,5.500000,0.812500,20.050000


In [25]:
train_df.corr()

,target,sum_amount,count_loans,freq_recharges_weekly,recharges_weekly,delta_after_recharges
target,1.000000,-0.231116,-0.269322,-0.290510,-0.211319,-0.088737
sum_amount,-0.231116,1.000000,0.975378,0.647190,0.485158,0.100453
count_loans,-0.269322,0.975378,1.000000,0.638603,0.464965,0.101892
freq_recharges_weekly,-0.290510,0.647190,0.638603,1.000000,0.712777,0.085745
recharges_weekly,-0.211319,0.485158,0.464965,0.712777,1.000000,0.127432
delta_after_recharges,-0.088737,0.100453,0.101892,0.085745,0.127432,1.000000


# SMOTE oversampling

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_tmp = train_df.reset_index()
df_tmp['index'] = le.fit_transform(df_tmp['uuid'])
train_X,train_y = oversample.fit_resample(
    df_tmp.drop(['uuid','target'],axis=1),
    df_tmp[['target']])
train_X['uuid'] = le.inverse_transform(train_X['index'])
train_X.set_index('uuid')
train_X.drop('index',axis=1,inplace=True)




test_df = train_X.merge(test_loans_fe[['uuid','target']],on='uuid',how='inner')
train_X.drop('uuid',axis=1,inplace=True)
test_df.drop('uuid',axis=1,inplace=True)

# Undersampling

In [26]:
from imblearn.under_sampling import RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=2022)
train_X, train_y = under_sampler.fit_resample(
    train_df.drop(['target'],axis=1),
    train_df[['target']])

test_df = train_X.merge(test_loans_fe[['uuid','target']],on='uuid',how='inner')
train_X.drop('uuid',axis=1,inplace=True)
test_df.drop('uuid',axis=1,inplace=True)

In [27]:
len(s.sample_indices_)

238

# Lembrar de fazer o a otimização dos parametros

# Com multicolinearidade

In [28]:
#fit classification model on t_df random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report


rfc = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2022)
rfc.fit(train_X, train_y)
y_pred = rfc.predict(test_df.drop(['target'],axis=1))
print('Accuracy: ',accuracy_score(test_df['target'], y_pred))
print('Confusion Matrix: \n',confusion_matrix(test_df['target'], y_pred))
print('Classification Report: \n',classification_report(test_df['target'], y_pred))


Accuracy:  0.5966386554621849
Confusion Matrix: 
 [[118  66]
 [ 30  24]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.64      0.71       184
           1       0.27      0.44      0.33        54

    accuracy                           0.60       238
   macro avg       0.53      0.54      0.52       238
weighted avg       0.68      0.60      0.63       238



/tmp/ipykernel_46910/3595253554.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(train_X, train_y)


# Sem multicolinearidade

In [29]:
train_X.corr()

,sum_amount,count_loans,freq_recharges_weekly,recharges_weekly,delta_after_recharges
sum_amount,1.000000,0.966692,0.652630,0.477287,0.219949
count_loans,0.966692,1.000000,0.653344,0.466286,0.242840
freq_recharges_weekly,0.652630,0.653344,1.000000,0.731600,0.215462
recharges_weekly,0.477287,0.466286,0.731600,1.000000,0.266321
delta_after_recharges,0.219949,0.242840,0.215462,0.266321,1.000000


In [97]:
#fit classification model on t_df random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline


#Ajust the model to pipeline
rfc = RandomForestClassifier()
rfc.fit(train_X.drop('count_loans',axis=1), train_y)
y_pred = rfc.predict(test_df.drop(['count_loans','target'],axis=1))
print('Accuracy: ',accuracy_score(test_df['target'], y_pred))
print('Confusion Matrix: \n',confusion_matrix(test_df['target'], y_pred))
print('Classification Report: \n',classification_report(test_df['target'], y_pred))

Accuracy:  0.5504201680672269
Confusion Matrix: 
 [[97 87]
 [20 34]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.53      0.64       184
           1       0.28      0.63      0.39        54

    accuracy                           0.55       238
   macro avg       0.56      0.58      0.52       238
weighted avg       0.70      0.55      0.59       238



/tmp/ipykernel_46910/3043307356.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(train_X.drop('count_loans',axis=1), train_y)


In [31]:
under_sampler = RandomUnderSampler(random_state=2022)
train_X, train_y = under_sampler.fit_resample(
train_df.drop(['target'],axis=1),
train_df[['target']])

test_df = train_X.merge(test_loans_fe[['uuid','target']],on='uuid',how='inner')
train_X.drop('uuid',axis=1,inplace=True)
test_df.drop('uuid',axis=1,inplace=True)

pipeline = Pipeline([
('rfc', RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2022)),
('under_sampler', RandomUnderSampler(random_state=2022)),
])
pipeline.fit(train_X, train_y)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestClassifier(max_depth=3, random_state=2022)' (type <class 'sklearn.ensemble._forest.RandomForestClassifier'>) doesn't

# Pycaret

In [ ]:
from pycaret.classification import *
s = setup(data=pd.concat([train_X, train_y], axis=1), target = 'target')

,Description,Value
0,session_id,6788
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(238, 6)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8559,0.8986,0.8278,0.8806,0.8430,0.7106,0.7237,0.0450
et,Extra Trees Classifier,0.8320,0.8635,0.7903,0.8806,0.8168,0.6624,0.6839,0.0400
gbc,Gradient Boosting Classifier,0.8316,0.8971,0.7903,0.8594,0.8126,0.6615,0.6754,0.0130
ada,Ada Boost Classifier,0.8202,0.8924,0.7528,0.8649,0.7977,0.6385,0.6521,0.0170
dt,Decision Tree Classifier,0.8085,0.8179,0.8153,0.8125,0.8020,0.6174,0.6373,0.0030
knn,K Neighbors Classifier,0.8018,0.8930,0.6903,0.9038,0.7550,0.5985,0.6303,0.0060
lightgbm,Light Gradient Boosting Machine,0.7952,0.9053,0.7778,0.8172,0.7833,0.5875,0.6052,0.0060
svm,SVM - Linear Kernel,0.7890,0.0000,0.7264,0.8841,0.7561,0.5722,0.6198,0.0030
lr,Logistic Regression,0.7353,0.8776,0.7764,0.7168,0.7359,0.4704,0.4853,0.2410
ridge,Ridge Classifier,0.7290,0.0000,0.8264,0.6921,0.7475,0.4605,0.4789,0.0030


In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…